# Azure OpenAI Assistants API - Streaming RAG Chat FastAPI

Notebook contains logic for testing an Azure OpenAI Assistants API deployment designed to answer questions from a RAG store (Azure AI Search Index). This notebook submits requests to either a local or cloud-hosted FastAPI endpoint which contains logic for creating threads, submitting messages, calling tools, and streaming chat responses. 

### Import required packages

In [1]:
import requests
import json
from IPython.display import display, HTML, Markdown, clear_output
import os

import threading
from dotenv import load_dotenv

load_dotenv(override=True)

uri = 'http://127.0.0.1:8000'

### Streaming display helper function

In [20]:
# Function to process and display the streamed response
def process_streamed_response(response):
    buffer = ''
    try:
        for line in response.iter_lines(decode_unicode=True):
            if line:
                buffer += line + '\n'
                clear_output(wait=True)
                display(Markdown(buffer))
               
            
        # Ensure the final content is displayed
        clear_output(wait=True)
        display(Markdown(buffer))
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        response.close()

### Workflow - Step #1: Create a Thread

Following request creates a new Assistants API-managed thread

In [19]:
response = requests.post(f"{uri}/create_thread")
thread_id = response.json()

### Workflow - Step #2: Submit User Query as New Message

New message is added to the thread which is subsequently run. The response is then streamed and displayed here inline.

In [ ]:
url = f"{uri}/run_assistant"

data={
    'thread_id': thread_id, 
    'message': 'What are my health care options?'
}

# Send the POST request with stream=True
response = requests.post(url, json=data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Start a separate thread to process the response
    thread = threading.Thread(target=process_streamed_response, args=(response,))
    thread.start()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

<i>Retrieving Documents...</i> {'keywords': 'health care options', 'document_count': 5} <br><br>


### Workflow - Step #3: Continue Conversation

Follow up with another question that builds on the initial question. Note that context is maintained via the Assistants API state management.

In [22]:
url = f"{uri}/run_assistant"

data={
    'thread_id': thread_id, 
    'message': 'What about dental?'
}

# Send the POST request with stream=True
response = requests.post(url, json=data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Start a separate thread to process the response
    thread = threading.Thread(target=process_streamed_response, args=(response,))
    thread.start()
else:
    print(f"Error: {response.status_code}")
    print(response.text)

<i>Retrieving Documents...</i> {'keywords': 'health care options', 'document_count': 3} <br><br>
<i>Retrieving Documents...</i> {'keywords': 'dental care options', 'document_count': 3} <br><br>


In [12]:
url = f"{uri}/run_assistant"

data={
    'thread_id': thread_id, 
    'message': 'How do I maintain all of this equipment?'
}

# Send the POST request with stream=True
response = requests.post(url, json=data, stream=True)

# Check if the request was successful
if response.status_code == 200:
    # Start a separate thread to process the response
    thread = threading.Thread(target=process_streamed_response, args=(response,))
    thread.start()
else:
    print(f"Error: {response.status_code}")
    print(response.text)
